In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('df_all_merged.csv')

In [7]:
df

,Unnamed: 0,userID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,diff_con_discon[min],diff_con_done[min],SoC_start[%],SoC_end[%],battery_capacity[kWh],battery_capacity[%]
0,0,1083,2019-03-31 12:40:43-07:00,2019-03-31 19:26:57-07:00,2019-03-31 19:26:52-07:00,6.219,406,406,22.63,100.00,8.03750,77.37
1,1,489,2019-03-31 14:16:46-07:00,2019-03-31 14:23:08-07:00,2019-03-31 14:23:03-07:00,0.562,6,6,45.43,46.27,67.22375,0.84
2,2,243,2019-03-31 16:07:33-07:00,2019-03-31 16:46:29-07:00,2019-03-31 16:46:21-07:00,3.870,38,38,43.07,88.65,8.49000,45.58
3,3,1093,2019-03-31 16:21:59-07:00,2019-03-31 18:41:26-07:00,2019-03-31 16:45:20-07:00,0.789,139,23,89.51,100.00,7.52375,10.49
4,4,365,2019-03-31 16:35:22-07:00,2019-03-31 17:55:10-07:00,2019-03-31 17:55:02-07:00,4.501,79,79,20.00,100.00,5.62625,80.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1730,1730,1152,2019-05-31 21:32:23-07:00,2019-06-01 09:27:29-07:00,2019-06-01 00:55:35-07:00,11.865,715,203,39.07,100.00,19.47375,60.93
1731,1731,675,2019-06-01 07:55:43-07:00,2019-06-01 09:44:17-07:00,2019-06-01 09:20:10-07:00,8.082,108,84,46.95,100.00,15.23500,53.05
1732,1732,1133,2019-06-01 09:59:26-07:00,2019-06-01 11:27:58-07:00,2019-06-01 11:03:24-07:00,6.062,88,63,38.57,100.00,9.86875,61.43
1733,1733,365,2019-06-01 10:45:34-07:00,2019-06-01 11:28:51-07:00,2019-06-01 11:28:42-07:00,2.502,43,43,55.53,100.00,5.62625,44.47


In [8]:
def current_kWh_for_location(df, timeline, start_time, end_time,SoC_start_column, battery_capacity_column, kWhDelivered_column ):
  car_parked = []
  parked_kWh_SoC = []

  j= -1
  for i in range(len(df)): #df
    j = j + 1
    car_parked = []
    parked_kWh_SoC = []
    parked_kWh_Max = []

    for i in range(len(timeline)): #timeline
      if start_time[j] <= timeline[i] and end_time[j] > timeline[i]: #start_time & end_time
        car_parked.append(1)
      else:
        car_parked.append(0)

    if sum(car_parked) > 0:
      SoC_start_holder = round(SoC_start_column[j]*battery_capacity_column[j]/100 ,2) #Soc_start_column * baterry_capacity_column/100
      kWh_holder = round(kWhDelivered_column[j]/sum(car_parked),2)# kWhDelivered_column & with sum(9car_parked) linear interpolation
      parked_kWh_SoC = [i* SoC_start_holder for i in car_parked] # multiplicate two lists
      parked_kWh_Max = [i* battery_capacity_column[j] for i in car_parked] # potential kWh capacity
  
      for i in range(len(parked_kWh_SoC)):
        if parked_kWh_SoC[i-1] != 0 and parked_kWh_SoC[i] != 0:
          parked_kWh_SoC[i] = parked_kWh_SoC[i-1] + kWh_holder

    else:
      parked_kWh_SoC = [0] * len(car_parked) #list add only with same len
      parked_kWh_Max = [0] *len(car_parked)

    if j <= 0: # how in for loop ?
      parked_cars_len = [0] * len(car_parked) #list add only with same len
      parked_kWh_SoC = [0] * len(car_parked)
      parked_kWh_Max = [0] *len(car_parked)

      sum_parked_count = [a + b for a, b in zip(car_parked, parked_cars_len)] # add list together
      sum_kWh_count = [a + b for a, b in zip(parked_kWh_SoC, parked_cars_len)]
      sum_kWh_potential = [a + b for a, b in zip(parked_kWh_Max, parked_cars_len)]
  
    else:
      sum_parked_count = [a + b for a, b in zip(car_parked, sum_parked_count)]
      sum_kWh_count = [a + b for a, b in zip(parked_kWh_SoC, sum_kWh_count)]
      sum_kWh_potential = [a + b for a, b in zip(parked_kWh_Max, sum_kWh_potential)]
  
  sum_kWh_count = [round(num, 2) for num in sum_kWh_count] # round to 2 digits
  sum_kWh_potential = [round(num, 2) for num in sum_kWh_potential] # round to 2 digits

  df_current_kWh = pd.DataFrame(list(zip(timeline,sum_parked_count,sum_kWh_count, sum_kWh_potential)),
                      columns = ['zeitreihe','sum_parked_count', 'sum_kWh_count','sum_kWh_potential'])
  return df_current_kWh

In [9]:
df_sum = current_kWh_for_location(df, zeitreihe_aware, connectionTime, disconnectTime, df_all_merged['SoC_start[%]'],df['battery_capacity[kWh]'],df_all_merged['kWhDelivered'])

NameError: name 'df_all_merged' is not defined